<a href="https://colab.research.google.com/github/drustantmetar/DataFrame1/blob/main/Dataframe_1_(Practical1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# load the libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [ ]:
# create spark session
sc : SparkContext = SparkSession\
.builder\
.appName("IMDB Analysis")\
.master('local')\
.getOrCreate()\
.sparkContext

In [ ]:
# Import datea
#movies=sc.textFile("movies.dat")
#from google.colab import files
#movies=files.upload()


In [56]:
# Create Rdd
movies=sc.textFile("/content/movies.dat");
rating=sc.textFile("/content/ratings.dat")

In [57]:
# split the string with double ::
def splitString(myfile):
  return myfile.split("::",4)

In [61]:
movies=sc.textFile("movies.dat").map(splitString);
rating=sc.textFile("ratings.dat").map(splitString)

Display text from datea sets

In [59]:
movies.take(3)

[['1', 'Toy Story (1995)', "Animation|Children's|Comedy"],
 ['2', 'Jumanji (1995)', "Adventure|Children's|Fantasy"],
 ['3', 'Grumpier Old Men (1995)', 'Comedy|Romance']]

In [60]:
rating.take(3)

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968']]

In [62]:
# Join the Rdd
joinRdd=movies.keyBy(lambda k : k[0]).join(rating.keyBy(lambda k: k[1]))

In [63]:
joinRdd.take(3)

[('3',
  (['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
   ['26', '3', '2', '978140022'])),
 ('3',
  (['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
   ['45', '3', '2', '977989159'])),
 ('3',
  (['3', 'Grumpier Old Men (1995)', 'Comedy|Romance'],
   ['62', '3', '3', '977904968']))]

In [67]:
#we need to get the movie names
# Explanation
'''
x=		[('5',  (['5', 'Father of the Bride Part II (1995)', 'Comedy'],   ['26', '5', '5', '978144881'])),
		('5',  (['5', 'Father of the Bride Part II (1995)', 'Comedy'],   ['48', '5', '3', '978013996'])),
		('5',  (['5', 'Father of the Bride Part II (1995)', 'Comedy'],   ['155', '5', '2', '977256084']))]
 =======================
		['5', 'Father of the Bride Part II (1995)', 'Comedy'],   ['26', '5', '5', '978144881']
 x[1]=[movie_data , ratings_data]
x[1][0]= movie_data   -> '5', 'Father of the Bride Part II (1995)', 'Comedy'
x[1][0][1]= 'Father of the Bride Part II (1995)'

'''
finalRdd=joinRdd.groupBy(lambda gbk : gbk[1][0][1])
finalRdd.take(3)
# It return movie name with some value with its so we need to take the length of that value

[('Father of the Bride Part II (1995)',
  <pyspark.resultiterable.ResultIterable at 0x7da8a5955850>),
 ('Heat (1995)', <pyspark.resultiterable.ResultIterable at 0x7da8a5cb5d50>),
 ('GoldenEye (1995)',
  <pyspark.resultiterable.ResultIterable at 0x7da8a5cb1050>)]

In [71]:
myRdd=finalRdd.map(lambda e : (e[0],len(e[1])))
myRdd.take(3)

[('Father of the Bride Part II (1995)', 296),
 ('Heat (1995)', 940),
 ('GoldenEye (1995)', 888)]